In [9]:
from dotenv import load_dotenv 
import sys,os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../../../src")))
load_dotenv()

%load_ext autoreload
%autoreload 2 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
from utils.custom_message_text import show_prompt
from core.prompts.PromptManager import PromptManager
from core.prompts.PromptType import PromptType
prompt_manager = PromptManager()
prompt = prompt_manager.get_prompt(PromptType.SYSTEM_RESEARCH, messages=[], date="")
show_prompt(prompt)

╭──────────────────────────────────────────────────── Prompt ─────────────────────────────────────────────────────╮
│                                                                                                                 │
│  <역할>                                                                                                         │
│  당신은 사용자의 입력 주제에 대해 리서치를 수행하는 연구 보조원 입니다.                                         │
│  오늘 날짜는 입니다. 오늘 날짜를 기반으로 리서치를 수행합니다.                                                  │
│                                                                                                                 │
│  <사고 절차>                                                                                                    │
│  인간 연구자처럼 사고 하며 다음 절차를 따르세요:                                                                │
│  1. 질문을 주의 깊게 읽고 핵심 요구를 파악                                                                      │
│  2. 먼저 포괄적인 검색을 수행                                                                                   │
│  3. 각 검색 후 결과 평가 ("충분한가?", "무엇이 부족한가?")                                                      │
│  4. 부족하면 더 구체적인 검색 수행                                                                              │
│  5. 충분하면 리서치 종료                                                                                        │
│                                                                                                                 │
│  <사용 가능한 도구들>                                                                                           │
│  - **arxiv_search**: 논문 사이트 arxiv api 검색용                                                               │
│  - **tavily_search**: 웹 검색용                                                                                 │
│  - **rag_search**: RAG 검색용                                                                                   │
│  - **think_tool**: 결과 분석 및 다음 단계 계획용                                                                │
│  각 검색 후 반드시 **think_tool**을 사용해 결과를 평가하세요.                                                   │
│                                                                                                                 │
│  <행동 규칙>                                                                                                    │
│  - 인간 연구자처럼 사고하며, 근거를 명확히 기술하세요.                                                          │
│  - 각 검색 후 반드시 주어진 주제에 대해 도구를 활용해 자료를 수집하고 분석하세요.                               │
│  - 너무 많은 검색 반복을 피하고, 판단 근거를 명확히 기술하세요.                                                 │
│                                                                                                                 │
│  <Hard Limits>                                                                                                  │
│  - 간단한 쿼리: 최대 3회 검색                                                                                   │
│  - 복잡한 쿼리: 최대 5회 검색                                                                                   │
│  - 5회 이상에도 답을 찾지 못하면 종료                                                                           │
│                                                                                                                 │
│  <작업>                                                                                                         │
│  주어진 주제에 대해 툴을 활용해 자료를 수집하고 분석하세요.                                                     │
│  툴은 순차 또는 병렬로 호출할 수 있으며, 루프 형태로 리서치를 수행합니다.                                       │
│                                                                                                                 │
│  <사고 과정을 보여주세요>                                                                                       │
│  각 검색 후 think_tool로 반성적 질문 수행:                                                                      │
│  - 무엇을 찾았는가?                                                                                             │


Writing research_state.py


In [ ]:
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage
from tavily import TavilyClient

from core.prompts.PromptManager import PromptManager
from core.prompts.PromptType import PromptType
from typing_extensions import Annotated, List, Literal

from research_state import Summary
from langchain_core.tools import tool, InjectedToolArg
from core.model.llm import gpt_4_1_mini_llm
from utils.helper import get_today_str


summary_model = gpt_4_1_mini_llm
tavily_client = TavilyClient()


def tavily_search_multiple(
    search_queries: List[str], max_results: int = 3, include_raw_content: bool = True
) -> List[dict]:
    search_docs = []
    for query in search_queries:
        result = tavily_client.search(
            query,
            max_results=max_results,
            include_raw_content=include_raw_content,
            topic="ai",
        )
        search_docs.append(result)
    return search_docs


prompt_manager = PromptManager()


def summary_web_content(web_content: str) -> str:

    parser_model = summary_model.with_structured_output(Summary)
    summary = parser_model.invoke(
        [
            HumanMessage(
                content=prompt_manager.get_prompt(
                    PromptType.SYSTEM_SUMMARY_WEB_CONTENT,
                    web_content=web_content,
                    date=get_today_str(),
                )
            )
        ]
    )
    try:
        formatted_summary = (
            f"<summary>\n{summary.summary}\n</summary>"
            f"<key_excerpts>\n{summary.key_excerpts}\n</key_excerpts>"
        )
    except:
        pass 
    
    return formatted_summary



# def deduplicate_search_results(search_results:List[dict]) -> dict:
#     unique_results = {}
#     for response in search_results:
#         for result in response['results']:
#             url = result['url']
#             if url not in unique_results:
#                 unique_results[url] = result

#     return unique_results

# def summary_search_results(unique_results:dict) -> dict:
#     summarized_results = {}

#     for url, result in unique_results.items():
#         if not result.get("raw_content"):
#             content = result['content']
#         else:
#             content = summary_web_content(result['raw_content'])

#         summarized_results[url] = {
#             'title':result['title'],
#             'content':content
#         }

#     return summarized_results

# def format_search_output(summarized_results:dict) -> str:
#     if not summarized_results:
#         return "유효한 검색 결과를 찾을 수 없습니다. 다른 검색어를 사용하거나 다른 검색 API를 이용해 보세요."

#     formatted_output = "Search results: \n\n"
#     for i, (url, result) in enumerate(summarized_results.items(), 1):
#         formatted_output += f"\n\n--- SOURCE {i}: {result['title']} ---\n"
#         formatted_output += f"URL: {url}\n\n"
#         formatted_output += f"SUMMARY: \n{result['content']}\n\n"
#         formatted_output += "-" * 80 + "\n"

#     return formatted_output

# @tool(parse_docstring=True)
# def tavily_search(
#     query:str,
#     max_results:Annotated[int,InjectedToolArg] = 3,
#     topic: Annotated[Literal["general","news","finance"], InjectedToolArg] = "general"
# ) -> str:
#     """Tavliy 검색 API에서 결과를 가져오고, 콘텐츠 요약을 수행합니다.

#     Args:
#         query: 실행할 단일 검색 쿼리
#         max_results: 반환할 최대 결과 수
#         topic: 결과를 필터링할 주제 ('general','news','finance')

#     Returns:
#         요약이 포함된 검색 결과를 포맷된 문자열 형태로 반환 합니다.
#     """

#     search_results = tavily_search_multiple(
#         [query], max_results=max_results, topic=topic, include_raw_content=True
#     )

#     unique_results = deduplicate_search_results(search_results)
#     summarized_results = summary_search_results(unique_results)
#     return format_search_output(summarized_results)

# @tool(parse_docstring=True)
# def think_tool(reflection:str) -> str:
#     """연구 진행 상황과 의사결정에 대한 전략적 성찰(Reflection)을 위한 도구입니다.

#     이 도구는 각 검색(search) 이후에 결과를 분석하고, 다음 단계를 쳬계적으로 계획하기 위해 사용합니다.
#     이는 연구 워크플로우 중 잠시 멈춰서 품질 높은 결정을 내리기 위한 의도적인 검토 단계를 만듭니다.

#     사용 시점:
#     - 검색 결과를 받은 후: 어떤 핵심 정보를 발견했는가?
#     - 다음 단계를 결정하기 전: 충분한 정보를 확보했는가?
#     - 연구의 공백을 평가할 때: 아직 어떤 중요한 정보가 부족한가?
#     - 연구를 마무리하기 전: 이제 완전한 답변을 제공할 수 있는가?

#     성찰(Reflection)은 다음을 포함해야 합니다:
#     1. 현재 발견 내용 분석 - 어떤 구체적 정보를 수집했는가?
#     2. 정보 공백 평가 - 아직 부족한 핵심 정보는 무엇인가?
#     3. 품질 평가 - 좋은 답변을 위해 충분한 근거/예시를 확보했는가?
#     4. 전략적 결정 - 추가 검색을 계속할 것인가, 아니면 답변을 제시할 것인가?

#     Args:
#         reflection: 연구 진행 상황, 발견 내용, 정보 공백, 다음 단계에 대한 상세한 성찰 내용

#     Returns:
#         의사결정을 위한 성찰이 기록되었음을 확인하는 메시지
#     """
#     return f"Reflection recorded: {reflection}"